In [1]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import torch
from src.Dataset import VITDataset
from src.VIT import VitModel
import torch.nn as nn
from src.Train import train_vit
import os

In [2]:
model = VitModel(version=1)

In [3]:
dataset = load_dataset("imagenet-1k",trust_remote_code=True)
train_dataset = dataset['train']
val_dataset = dataset['validation']

Loading dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/25 [00:00<?, ?it/s]

In [4]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def preprocess_val(data):
    images = [image.convert('RGB') for image in data['image']]
    tensor_images = [train_transform(image).to('cuda') for image in images]
    data['image'] = torch.stack(tensor_images)
    return data

def preprocess_train(data):
    images = [image.convert('RGB') for image in data['image']]
    tensor_images = [train_transform(image).to('cuda') for image in images]
    data['image'] = torch.stack(tensor_images)
    return data

In [5]:
print(train_dataset)

Dataset({
    features: ['image', 'label'],
    num_rows: 1281167
})


In [6]:
train_dataset = VITDataset(dataset['train'], train_transform)
val_dataset = VITDataset(dataset['validation'], val_transform)

c:\Users\admin\anaconda3\envs\torch-cuda121\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
train_loader = DataLoader(train_dataset, batch_size= 64, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size= 64, shuffle = False)

In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
his = train_vit(1,model,train_loader,optimizer,version=1)

cuda


  0%|                  | 1/20019 [00:02<12:57:44,  2.33s/it]

loss=> 0:0.0003450560907367617


  0%|                  | 86/20019 [02:11<8:27:52,  1.53s/it]


KeyboardInterrupt: 

In [ ]:
if not os.path.exists(os.path.join('Model','VIT', 'v_1')):
        os.makedirs(os.path.join('Model','VIT', 'v_1'))
model.model.save_pretrained(os.path.join('Model','VIT', 'v_1'))